# Assignment 4: Training Transformers in PyTorch

*Author:* Thomas Adler

*Copyright statement:* This  material,  no  matter  whether  in  printed  or  electronic  form,  may  be  used  for  personal  and non-commercial educational use only.  Any reproduction of this manuscript, no matter whether as a whole or in parts, no matter whether in printed or in electronic form, requires explicit prior acceptance of the authors.

In this assignment we will implement and train a small transformer model and compare it to the LSTM in the previous assignment. 

## Exercise 1: Causal Self-Attention

Write a class named `CausalSelfAttention` that derives from `nn.Module` and whose `__init__` method takes (apart from the trivial `self`) one argument `hidden_size`. Implement a method `forward` that takes an input sequence `x` of shape $(N, T, D)$ (where $N$ is batch size, $T$ is sequence length, $D$ is hidden size) and performs scaled dot-product self-attention, i.e., 
$$
Y = \operatorname{softmax}\left(\frac{1}{\sqrt{D}} Q K^\top\right) V,
$$
where $Q = X W_Q$ and $K = X W_K$ and $V = X W_V$ and $X \in \mathbb{R}^{T \times D}$ and $W_Q, W_K, W_V \in \mathbb{R}^{D \times D}$ and softmax is applied in a row-wise manner and neglecting bias units. 
It is called self-attention because $Q, K, V$ are all computed from the same input $X$, which hence attends to itself. 

To have the attention be *causal* we need to make sure that we do not allow peeks into the future. That is, the output at time $t$ must be a function of the input at times $1, \dots, t$ but no further. The score matrix $E = \frac{1}{\sqrt{D}} Q K^\top$ has a shape of $T \times T$ and the entry $e_{ij}$ measures how strong the query at time $i$ attends to the key at time $j$. Therefore, positions where $j > i$ constitute peeks into the future and we have to set the corresponding attention values (i.e., the softmax-activated score) to zero. We can do that by setting the corresponding score to `float('-inf')`, which has the advantage that the normalization is adjusted automatically by the softmax. 

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
import time

class CasualSelfAttention(nn.Module):
    def __init__(self, hidden_size):
        self.hidden_size = hidden_size

        self.W_Q = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_K = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_V = nn.Linear(hidden_size, hidden_size, bias=False)
        
    def forward(self, x):
        """
       x: Input tensor of shape (N, T, D),
          where N = batch size, T = sequence length, D = hidden size.
       """
        N, T, D = x.shape
        
        # ensure hidden size matches input size
        assert D == self.hidden_size, "Input hidden size does not match initialized hidden size."

        Q = self.W_Q(x)  # Shape: (N, T, D)
        K = self.W_K(x)  # Shape: (N, T, D)
        V = self.W_V(x)  # Shape: (N, T, D)

        scale = 1 / (D ** 0.5)
        scores = torch.matmul(Q, K.transpose(-1, -2)) * scale

        causal_mask = torch.triu(torch.ones(T, T), diagonal=1).to(x.device) 
        scores = scores.masked_fill(causal_mask == 1, float('-inf')) # set the future tokens to -inf for softmax

        attention_weights = F.softmax(scores, dim=-1)  # Shape: (N, T, T)

        output = torch.matmul(attention_weights, V)  # Shape: (N, T, D)

        return output
        


## Exercise 2: Multi-Head Attention

Write a class `MultiHeadCausalSelfAttention` that derives from `nn.Module` and extends the functionality of `CausalSelfAttention` from the previous exercise. 
The `__init__` method takes arguments `hidden_size, n_head, dropout`. `n_head` specifies the number of attention heads and `dropout` specifies the intensity for the dropout layers. 
The `forward` method should split the hidden dimension of the pre-activations (i.e., $Q, K, V$) in `n_head` equally sized parts and perform attention to these parts in parallel. 
Apply the first dropout layer direcly after the softmax. 
After the multiplication of the scores with the values, recombine the output of the distinct attention heads back into a single hidden dimension of size $D$, i.e., the resulting shape should be the shape of the input. 
Then perform an additional output projection again resulting in a hidden dimension of $D$. 
Finally, apply the second dropout layer after the output projection. 

In [2]:
class MultiHeadCasualSelfAttention(nn.Module):
    def __init__(self, hidden_size, n_head, dropout):
        self.hidden_size = hidden_size
        self.n_head = n_head
        self.dropout = nn.Dropout(dropout)

        assert hidden_size % n_head == 0, "hidden_size must be divisible by n_head"
        self.head_dim = hidden_size // n_head

        # Learnable weight matrices for Q, K, V
        self.W_Q = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_K = nn.Linear(hidden_size, hidden_size, bias=False)
        self.W_V = nn.Linear(hidden_size, hidden_size, bias=False)

        # Output projection
        self.W_O = nn.Linear(hidden_size, hidden_size, bias=False)

    def forward(self, x):
        """
        x: Input tensor of shape (N, T, D),
           where N = batch size, T = sequence length, D = hidden size.
        """
        N, T, D = x.shape

        # Ensure the hidden size matches input dimensionality
        assert D == self.hidden_size, "Input hidden size does not match initialized hidden size."

        # Compute Q, K, V matrices and split into heads
        Q = self.W_Q(x).view(N, T, self.n_head, self.head_dim).transpose(1, 2)  # Shape: (N, n_head, T, head_dim)
        K = self.W_K(x).view(N, T, self.n_head, self.head_dim).transpose(1, 2)  # Shape: (N, n_head, T, head_dim)
        V = self.W_V(x).view(N, T, self.n_head, self.head_dim).transpose(1, 2)  # Shape: (N, n_head, T, head_dim)

        # Compute scaled dot-product attention scores
        scale = 1 / (self.head_dim ** 0.5)  # Scaling factor
        scores = torch.matmul(Q, K.transpose(-1, -2)) * scale  # Shape: (N, n_head, T, T)

        # Apply causal mask
        causal_mask = torch.triu(torch.ones(T, T), diagonal=1).to(x.device)  # Upper triangular mask
        scores = scores.masked_fill(causal_mask == 1, float('-inf'))

        # Apply softmax to scores (row-wise)
        attention_weights = F.softmax(scores, dim=-1)  # Shape: (N, n_head, T, T)
        attention_weights = self.dropout(attention_weights)  # Apply dropout

        # Weighted sum of values (V)
        output = torch.matmul(attention_weights, V)  # Shape: (N, n_head, T, head_dim)

        # Concatenate heads and project back to hidden size
        output = output.transpose(1, 2).contiguous().view(N, T, D)  # Shape: (N, T, D)
        output = self.W_O(output)  # Output projection
        output = self.dropout(output)  # Apply dropout

        return output

Linear(in_features=5, out_features=5, bias=False)


## Exercise 3: Multi-Layer Perceptron

Write a class `MLP` that derives from `nn.Module` and whose `__init__` method takes two arguments: `hidden_size` and `dropout`. 
It should implement a 2-layer feedforward network with `hidden_size` inputs, `4*hidden_size` hiddens, and `hidden_size` outputs. 
It should apply the GELU activation function to the hiddens and dropout to the outputs. 

In [ ]:

class MLP(nn.Module):
    def __init__(self, hidden_size, dropout):
        """
        Initializes a 2-layer feedforward network.
        
        Args:
            hidden_size (int): The size of the input and output features.
            dropout (float): Dropout probability to be applied to the output.
        """
        super(MLP, self).__init__()

        # Define the two linear layers
        self.fc1 = nn.Linear(hidden_size, 4 * hidden_size)  # First layer expands to 4*hidden_size
        self.fc2 = nn.Linear(4 * hidden_size, hidden_size)  # Second layer reduces back to hidden_size

        self.gelu = nn.GELU()

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """
        Forward pass through the MLP.

        Args:
            x (torch.Tensor): Input tensor of shape (batch_size, ..., hidden_size).
        
        Returns:
            torch.Tensor: Output tensor of shape (batch_size, ..., hidden_size).
        """
        # Apply the first linear layer and GELU activation
        x = self.gelu(self.fc1(x))

        # Apply the second linear layer and dropout
        x = self.dropout(self.fc2(x))

        return x


## Exercise 4: Block

Write a class `Block` that derives from `nn.Module` and whose `__init__` method takes arguments `hidden_size, n_head, dropout`. 
It should apply `nn.LayerNorm`, `CausalMultiHeadSelfAttention`, `nn.LayerNorm`, `MLP` in that order and feature residual connections from the input to the output of `CausalMultiHeadSelfAttention` and from there to the output of `MLP`. 

In [ ]:

class Block(nn.Module):
    def __init__(self, hidden_size, n_head, dropout):
        """
        A transformer block with layer normalization, multi-head attention, and MLP.
        """
        super(Block, self).__init__()
        self.hidden_size = hidden_size

        # Components of the Block
        self.ln1 = nn.LayerNorm(hidden_size)  # LayerNorm before attention
        self.attn = MultiHeadCasualSelfAttention(hidden_size, n_head, dropout)  # Multi-head attention
        self.ln2 = nn.LayerNorm(hidden_size)  # LayerNorm before MLP
        self.mlp = MLP(hidden_size, dropout)  # Feedforward network

    def forward(self, x):
        """
        Forward pass with residual connections.
        """
        # Residual connection from input to attention output
        attn_output = self.attn(self.ln1(x))  # Apply LayerNorm and attention
        x = x + attn_output  # Add residual

        # Residual connection from attention output to MLP output
        mlp_output = self.mlp(self.ln2(x))  # Apply LayerNorm and MLP
        x = x + mlp_output  # Add residual

        return x


## Exercise 5: GPT

Write a class `GPT` that derives from `nn.Module` and whose `__init__` method takes arguments `vocab_size, context_size, hidden_size, n_layer, n_head, dropout`. 
The `forward` method should take two arguments `x, y` representing sequences of input and target tokens, respectively, both of which have type `torch.long` and shape ($N$, $T$), and returns logits and loss as a tuple. 
The `GPT` module should feature two `nn.Embedding` layers, one for token embeddings and one for positional embedding, i.e., it should embed the position of the corresponding token within the input sequence. 
The positional embedding is necessary for the Transformer to determine the order of its inputs. 
Add the two embeddings and apply a dropout layer. 
Next, apply `n_layers` layers of `Block`s followed by a `nn.LayerNorm` and a `nn.Linear` (without bias) mapping to an output dimension of `vocab_size`. 
Finally, apply the cross-entropy loss function to the logits. 
To save some parameters, apply weight tying between the token embedding layer and the output layer, i.e., they should use the same weights. 
Initialize all weights using a normal distribution with a mean of zero and a standard deviation of 0.02 (except for the output layers of the `MLP`s use $0.02/\sqrt{2 * \mathtt{n\_layer}}$) and all biases to zero. 
Use the argument `dropout` as intensity for all dropout layers in the network. 

In [ ]:
########## YOUR SOLUTION HERE ##########

## Exercise 6: Optimizer

Add a method `configure_optimizers` to the class `GPT` that takes arguments `weight_decay, learning_rate, betas`. 
Divide the model parameters into two groups. 
The first group consists of all parameters with at least 2 dimensions, e.g., weight/embedding matrices and uses a decay of `weight_decay`. 
The second group consists of all other parameters, e.g., biases and layer norms, and does not use weight decay.
Construct and return a `torch.optim.AdamW` optimizer with `learning_rate` and `betas` that operates on these two parameter groups. 

In [ ]:
########## YOUR SOLUTION HERE ##########

## Exercise 7: Training

In the code cell below you find some globals, helper functions, and boilerplate code. Extend the given code by a training loop that 
* stops after `max_iters` iterations
* applies the learning rate schedule implemented in `get_lr`
* applies gradient clipping at `grad_clip` using `torch.nn.utils.clip_grad_norm_`
* accumulates gradients for `gradient_accumulation_steps` batches before each weight update
* logs the training loss and learning rate every `log_interval` iterations
* evaluates (and potentially checkpoints) the model using `estimate_loss` every `eval_iters` iterations.

The provided hyperparameter values should be a good guess for training a tiny model on CPU but feel free to experiment with them as you please. In particular, if you have a GPU available, you can try to scale things up a bit. 

In [ ]:
eval_interval = 250 # validate model every .. iterations
log_interval = 10 # log training loss every .. iterations
eval_iters = 20 # number of batches for loss estimation
gradient_accumulation_steps = 5 * 8 # used to simulate larger training batch sizes
batch_size = 12 # if gradient_accumulation_steps > 1, this is the micro-batch size
context_size = 64 # sequence length
vocab = 'abcdefghijklmnopqrstuvwxyz0123456789 .!?' # vocabulary
vocab_size = len(vocab) # 40
n_layer = 4 # number of layers
n_head = 4 # number of attention heads
hidden_size = 128 # layer size
dropout = 0.0 # for pretraining 0 is good, for finetuning try 0.1+
learning_rate = 1e-3 # max learning rate
max_iters = 2000 # total number of training iterations
weight_decay = 1e-1
beta1 = 0.9 # for AdamW
beta2 = 0.99 # for AdamW
grad_clip = 1.0 # clip gradients at this value, or disable with 0.0
warmup_iters = 100 # how many steps to warm up for
min_lr = 1e-4 # minimum learning rate, usually ~= learning_rate/10

# learning rate decay scheduler (cosine with warmup)
def get_lr(it):
    # 1) linear warmup for warmup_iters steps
    if it < warmup_iters:
        return learning_rate * it / warmup_iters
    # 2) if it > max_iters, return min learning rate
    if it > max_iters:
        return min_lr
    # 3) in between, use cosine decay down to min learning rate
    decay_ratio = (it - warmup_iters) / (max_iters - warmup_iters)
    assert 0 <= decay_ratio <= 1
    coeff = 0.5 * (1.0 + math.cos(math.pi * decay_ratio)) # coeff ranges 0..1
    return min_lr + coeff * (learning_rate - min_lr)

def load_data(split):
    import re
    
    with open(f'trump_{split}.txt', 'r') as f:
        text = f.read()
    
    text = text.lower() # convert to lower case
    text = re.sub('[^a-z0-9 .!?]', ' ', text) # replace all unknown chars with ' '
    text = re.sub(' +', ' ', text) # reduce multiple blanks to one
    text = [vocab.index(t) for t in text]
    text = torch.tensor(text, dtype=torch.long)
    return text
    
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - context_size, (batch_size,))
    x = torch.stack([data[i:i+context_size] for i in ix])
    y = torch.stack([data[i+1:i+1+context_size] for i in ix])
    return x, y

# helps estimate an arbitrarily accurate loss over either split using many batches
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# data, model, optimizer, etc. 
train_data = load_data('train')
val_data = load_data('val')
model = GPT(vocab_size, context_size, hidden_size, n_layer, n_head)
optimizer = model.configure_optimizers(weight_decay, learning_rate, (beta1, beta2))
iter_num = 0
best_val_loss = 1e9
X, Y = get_batch('train') # fetch the very first batch
t0 = time.time()

########## YOUR SOLUTION HERE ##########

## Exercise 8: Inference

Add a method `generate` to the class `GPT` that takes arguments `x, max_new_tokens, temperature=1.0`. 
The method should take a batch of token sequences `x`, which it should extend by `max_new_tokens` new tokens generated by the model. 
Once you have computed the logits for the next token, divide them by `temperature` before applying the softmax. 
After applying the softmax, sample the next token from the resulting categorical distribution. 
Try out different values for `temperature` and compare the results to those from the previous assignment. 

In [ ]:
########## YOUR SOLUTION HERE ##########